## Prepare images for training

Resize images to 64 x 64 x 1. For simplicity's sake, only use single channel from RGB images.

In [1]:
from matplotlib import image
from os import listdir
from PIL import Image
import numpy as np
from numpy import expand_dims


def load_samples():
    loaded_img = []

    for file in listdir('C:/Users/Nitro/Documents/pokegan/pokemon_jpg/pokemon_jpg'):
        img = Image.open('C:/Users/Nitro/Documents/pokegan/pokemon_jpg/pokemon_jpg/' + file)
        red, green, blue = img.split()
        img_resized = red.resize((64, 64))
        im2arr = np.array(img_resized)
        loaded_img.append(im2arr)

    X = np.asarray(loaded_img)
    X = expand_dims(X, axis=-1)
    X = X.astype('float32')
    X = X / 255.0
    return X

## Produce discriminator architecture

Use Leaky ReLU activation function after convolutions to reduce vanishing gradient problem to ensure better training for kernels. Additionally use Leaky ReLU  to add non-linearity to improve variance in learned kernels and allow model to adapt to various features.

Apply dropout layers as well to possibly remove some outputs and reduce overfitting.

Discriminator compresses images down into feature space. It is trained on real and fake images; therefore model is a convolutional neural network for binary-classification. 

In [2]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model

def discriminator(in_shape=(64, 64, 1)):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(4, 4), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

model = discriminator()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 64)        640       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 16, 16, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

## Load in all pokemon image samples

In [3]:
trainX = load_samples()
trainX.shape

(819, 64, 64, 1)

## Produce real-image dataset for training discriminator

Select random images from existing dataset and assign real class label

In [4]:
from numpy import ones
from numpy import zeros
from numpy.random import rand
from numpy.random import randint

def generate_real_samples(dataset, n):
    ind = randint(0, dataset.shape[0], n)
    X = dataset[ind]
    y = ones((n, 1))
    return X, y

## Produce fake-image dataset for training discriminator

Create pixel-space for however many image samples are selected. Then transform the pixel space into n random images and provide a class label.

In [5]:
def generate_fake_samples(n):
    X = rand(64 * 64 * n)
    X = X.reshape((n, 64, 64, 1))
    y = zeros((n, 1))
    return X, y

## Train discriminator

Adjust kernel weights per each batch then check prediction accuracy. Train and test splits handled in background by function, train_on_batch().

Keras by default ignores first dimension of input, therefore even though fake and real samples have shape (n, 64, 64, 1), Keras will read it in as (64, 64, 1) and work correctly.

Additionally train_on_batch() is a specialised training algorithm as opposed to fit(). Training in batches reduces memory overhead and reduces processing time. 

Notice fast convergence of algorithm. Due to strongly randomized fake images it works quickly, however later when discriminator competes with generator, training time will increase.

In [6]:
def train_discriminator(model, dataset, n_iter=50, n_batch=128):
    half_batch = int(n_batch/2)
    
    for i in range(n_iter):
        X_real, y_real = generate_real_samples(dataset, half_batch)
        _, real_acc = model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(half_batch)
        _, fake_acc = model.train_on_batch(X_fake, y_fake)
        print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))

model = discriminator()
train_discriminator(model, trainX)

>1 real=61% fake=16%
>2 real=77% fake=11%
>3 real=88% fake=8%
>4 real=89% fake=6%
>5 real=88% fake=5%
>6 real=97% fake=3%
>7 real=95% fake=3%
>8 real=100% fake=5%
>9 real=98% fake=3%
>10 real=100% fake=0%
>11 real=100% fake=2%
>12 real=100% fake=3%
>13 real=100% fake=3%
>14 real=100% fake=2%
>15 real=97% fake=6%
>16 real=100% fake=0%
>17 real=100% fake=2%
>18 real=100% fake=3%
>19 real=100% fake=2%
>20 real=100% fake=5%
>21 real=100% fake=2%
>22 real=100% fake=0%
>23 real=100% fake=0%
>24 real=100% fake=3%
>25 real=100% fake=2%
>26 real=100% fake=5%
>27 real=100% fake=0%
>28 real=100% fake=3%
>29 real=100% fake=2%
>30 real=100% fake=2%
>31 real=100% fake=0%
>32 real=100% fake=2%
>33 real=100% fake=5%
>34 real=100% fake=5%
>35 real=100% fake=5%
>36 real=100% fake=2%
>37 real=100% fake=6%
>38 real=100% fake=23%
>39 real=100% fake=22%
>40 real=100% fake=42%
>41 real=100% fake=25%
>42 real=100% fake=34%
>43 real=100% fake=64%
>44 real=100% fake=58%
>45 real=100% fake=64%
>46 real=100% fake